In [1]:
from keras import backend as K
from keras.optimizers import SGD
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.
/Users/vkhanna/Documents/GitHub/Keras_SmileFace_Challenge/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/vkhanna/Documents/GitHub/Keras_SmileFace_Challenge/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/vkhanna/Documents/GitHub/Keras_SmileFace_Challenge/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) /

In [2]:
# Could use larger dimensions, but will make training
# times much much longer
img_width, img_height = (75, 75)

In [3]:
train_dir = 'FEI/train'
test_dir = 'FEI/test'

epochs = 40
batch_size = 8
num_train_samples = 320
num_test_samples  =  80

# perform random transformations so that the
# data is more varied
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen  = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='binary',
                                                    shuffle=True)

validation_generator = test_datagen.flow_from_directory(test_dir,
                                                        target_size=(img_width, img_height),
                                                        batch_size=batch_size,
                                                        class_mode='binary',
                                                        shuffle=True)

Found 320 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [4]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [5]:
input_shape

(75, 75, 3)

In [9]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), activation="relu", input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolution is done, so make the fully connected layer
model.add(Flatten())

# Drop 50% of the neurons
model.add(Dense(25, activation="relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# compilation
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [10]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

model.fit_generator(train_generator,
                    steps_per_epoch=num_train_samples // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=num_test_samples // batch_size,
                    callbacks=[early_stopping])

Epoch 1/40
40/40 [==============================] - 3s 67ms/step - loss: 0.6963 - acc: 0.5188 - val_loss: 0.6968 - val_acc: 0.4625
Epoch 2/40
40/40 [==============================] - 2s 60ms/step - loss: 0.6911 - acc: 0.5656 - val_loss: 0.6867 - val_acc: 0.7000
Epoch 3/40
40/40 [==============================] - 2s 62ms/step - loss: 0.6832 - acc: 0.5719 - val_loss: 0.6866 - val_acc: 0.5250
Epoch 4/40
40/40 [==============================] - 3s 64ms/step - loss: 0.6805 - acc: 0.5625 - val_loss: 0.6560 - val_acc: 0.6375
Epoch 5/40
40/40 [==============================] - 2s 58ms/step - loss: 0.6748 - acc: 0.6000 - val_loss: 0.6145 - val_acc: 0.6125
Epoch 6/40
40/40 [==============================] - 2s 61ms/step - loss: 0.6665 - acc: 0.6156 - val_loss: 0.5850 - val_acc: 0.7750
Epoch 7/40
40/40 [==============================] - 2s 61ms/step - loss: 0.6662 - acc: 0.5906 - val_loss: 0.6774 - val_acc: 0.6250
Epoch 8/40
40/40 [==============================] - 3s 64ms/step - loss: 0.6754 - a